In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import json
import numpy as np
from PIL import Image


model_fp32 = models.resnet18(pretrained=True).to('cpu')
#print(list(model_fp32.children()))

model_fp32.eval()

fused_model = torch.quantization.fuse_modules(model_fp32, [
    ['conv1', 'bn1'], 
    ['layer1.0.conv1', 'layer1.0.bn1'],
    ['layer1.0.conv2', 'layer1.0.bn2'],
    ['layer1.1.conv1', 'layer1.1.bn1'],
    ['layer1.1.conv2', 'layer1.1.bn2'],
    ['layer2.0.conv1', 'layer2.0.bn1'],
    ['layer2.0.conv2', 'layer2.0.bn2'],
    ['layer2.0.downsample.0', 'layer2.0.downsample.1'],
    ['layer2.1.conv1', 'layer2.1.bn1'],
    ['layer2.1.conv2', 'layer2.1.bn2'],
    ['layer3.0.conv1', 'layer3.0.bn1'],
    ['layer3.0.conv2', 'layer3.0.bn2'],
    ['layer3.0.downsample.0', 'layer3.0.downsample.1'],
    ['layer3.1.conv1', 'layer3.1.bn1'],
    ['layer3.1.conv2', 'layer3.1.bn2'],
    ['layer4.0.conv1', 'layer4.0.bn1'],
    ['layer4.0.conv2', 'layer4.0.bn2'],
    ['layer4.0.downsample.0', 'layer4.0.downsample.1'],
    ['layer4.1.conv1', 'layer4.1.bn1'],
    ['layer4.1.conv2', 'layer4.1.bn2']
], inplace=True)

print(fused_model)

print(torch.__version__)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): Identity()
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 

In [4]:
# SAVE MODEL AS ONNX
import torch
dummy_input=torch.randn(1, 3,224,224)
input_names = [ "input"]
output_names = [ "output" ]

torch.onnx.export(fused_model, dummy_input, "./resnet18_fusedmodel.onnx", verbose=True,input_names=input_names,output_names=output_names)

graph(%input : Float(1:150528, 3:50176, 224:224, 224:1, requires_grad=0, device=cpu),
      %conv1.weight : Float(64:147, 3:49, 7:7, 7:1, requires_grad=1, device=cpu),
      %conv1.bias : Float(64:1, requires_grad=1, device=cpu),
      %layer1.0.conv1.weight : Float(64:576, 64:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %layer1.0.conv1.bias : Float(64:1, requires_grad=1, device=cpu),
      %layer1.0.conv2.weight : Float(64:576, 64:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %layer1.0.conv2.bias : Float(64:1, requires_grad=1, device=cpu),
      %layer1.1.conv1.weight : Float(64:576, 64:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %layer1.1.conv1.bias : Float(64:1, requires_grad=1, device=cpu),
      %layer1.1.conv2.weight : Float(64:576, 64:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %layer1.1.conv2.bias : Float(64:1, requires_grad=1, device=cpu),
      %layer2.0.conv1.weight : Float(128:576, 64:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %layer2.0.conv1.bias : F

In [12]:
# INFERENCE
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import json
import numpy as np
from PIL import Image

class_index = json.load(open('imagenet_class_index.json', 'r'))

correct_top1 = 0
correct_top5 = 0
invalid = 0
invalid_wnid = []

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225])

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

def my_preprocess(filename):
    img = Image.open(filename)
    h, w = img.size
    if h > w:
        img = img.resize((256 * h // w , 256))
    else:
        img = img.resize((256 , 256 * w // h))
    h, w = img.size
    img = img.crop(( (h-224)/2, (w-224)/2, 224+(h-224)/2, 224+(w-224)/2))
    np_img = np.array(img)
    np_img = np_img / 255
    return (np_img - np.array([0.485, 0.456, 0.406]) ) / np.array([0.229, 0.224, 0.225])


fused_model.eval()
for index in range(1000):
    file = './Imagenet/' + str(index) + '.jpg'
    try:
        img = Image.open(file)
        img_tensor = preprocess(img)
        
        #img = my_preprocess(file)
        #img = img.transpose(2,0,1)
        #img_tensor = torch.from_numpy(img.astype(np.float32)).clone()
        
        img_tensor.unsqueeze_(0)

        outputs = fused_model(Variable(img_tensor).to('cpu'))

        softmax = torch.nn.Softmax(dim=1)
        softmax_out = softmax(outputs)

        for i in range(5):
            imagenet_key = str(softmax_out.topk(5)[1][0][i].item())
            if imagenet_key == str(index):
                if i == 0:
                    correct_top1 += 1
                correct_top5 += 1
        if index % 10 == 0:
            print(str(correct_top5) + ' / ' + str(index))
    except:
        invalid += 1
        #print(class_index[str(index)][0])
        invalid_wnid.append(class_index[str(index)][0])

print('TOP1 ACCURACY: ' + str(correct_top1) + ' / ' + str(1000-invalid))
print('TOP5 ACCURACY: ' + str(correct_top5) + ' / ' + str(1000-invalid))
print('INVALID DATA: ' + str(invalid))
print(invalid_wnid)


0.011967400000003181
1 / 0
0.008732799999961571
0.007237199999963195
0.00921520000019882
0.010925100000122256
0.011676200000010795
0.010943900000029316
0.00959370000009585
0.013730699999996432
0.011224300000094445
TOP1 ACCURACY: 9 / 1000
TOP5 ACCURACY: 10 / 1000
INVALID DATA: 0
[]


In [2]:
# SHOW MODEL PARAMS
import numpy as np
for key in fused_model.state_dict():
    print(key.ljust(35), end=': ')
    print(np.array(fused_model.state_dict()[key].shape), end=',  max=')
    print( max(np.ravel(np.array(fused_model.state_dict()[key]))), end='  min=')
    print( min(np.ravel(np.array(fused_model.state_dict()[key]))))
    print('')

conv1.weight                       : [64  3  7  7],  max=0.39387545  min=-0.30834106

conv1.bias                         : [64],  max=0.69328916  min=-0.64016706

layer1.0.conv1.weight              : [64 64  3  3],  max=0.33263963  min=-0.37450415

layer1.0.conv1.bias                : [64],  max=1.1022106  min=-0.8196391

layer1.0.conv2.weight              : [64 64  3  3],  max=0.52512556  min=-0.77078116

layer1.0.conv2.bias                : [64],  max=1.7853346  min=-1.2511991

layer1.1.conv1.weight              : [64 64  3  3],  max=0.27995738  min=-0.26628444

layer1.1.conv1.bias                : [64],  max=1.2079195  min=-1.0184617

layer1.1.conv2.weight              : [64 64  3  3],  max=0.7608536  min=-1.0473908

layer1.1.conv2.bias                : [64],  max=1.0843931  min=-1.1705731

layer2.0.conv1.weight              : [128  64   3   3],  max=0.21269578  min=-0.14254896

layer2.0.conv1.bias                : [128],  max=0.74007654  min=-0.43975008

layer2.0.conv2.weight      

In [3]:
import numpy as np
for key in fused_model.state_dict():
    print(key)
    np.savetxt('./resnet18_param/'+key+'.txt', np.ravel(np.array(fused_model.state_dict()[key])))

conv1.weight
conv1.bias
layer1.0.conv1.weight
layer1.0.conv1.bias
layer1.0.conv2.weight
layer1.0.conv2.bias
layer1.1.conv1.weight
layer1.1.conv1.bias
layer1.1.conv2.weight
layer1.1.conv2.bias
layer2.0.conv1.weight
layer2.0.conv1.bias
layer2.0.conv2.weight
layer2.0.conv2.bias
layer2.0.downsample.0.weight
layer2.0.downsample.0.bias
layer2.1.conv1.weight
layer2.1.conv1.bias
layer2.1.conv2.weight
layer2.1.conv2.bias
layer3.0.conv1.weight
layer3.0.conv1.bias
layer3.0.conv2.weight
layer3.0.conv2.bias
layer3.0.downsample.0.weight
layer3.0.downsample.0.bias
layer3.1.conv1.weight
layer3.1.conv1.bias
layer3.1.conv2.weight
layer3.1.conv2.bias
layer4.0.conv1.weight
layer4.0.conv1.bias
layer4.0.conv2.weight
layer4.0.conv2.bias
layer4.0.downsample.0.weight
layer4.0.downsample.0.bias
layer4.1.conv1.weight
layer4.1.conv1.bias
layer4.1.conv2.weight
layer4.1.conv2.bias
fc.weight
fc.bias


In [2]:
# REMOVE LAST LAYER FROM MODEL
import numpy as np
from torch import nn

remove_layer = 1

modules = list(fused_model.children())[:-remove_layer]
check_model = nn.Sequential(*modules)
#check_model = fused_model

print(check_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): Identity()
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 

In [4]:
# CHECK RESULT
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import json
import numpy as np
from PIL import Image

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225])

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

index = 1

check_model.eval()

file = './Imagenet/' + str(index) + '.jpg'
  
img = Image.open(file)
img_tensor = preprocess(img)
img_tensor.unsqueeze_(0)
#print(img_tensor.shape)
#print(img_tensor)

start = time.time()
outputs = check_model(Variable(img_tensor))
end = time.time()
print('Elapsed Time: ' + str(end-start))
print(outputs.shape)
print(outputs)

softmax = torch.nn.Softmax(dim=1)
softmax_out = softmax(outputs)
#print(softmax_out.shape)
#print(softmax_out)

print(softmax_out.topk(5)[1][0])

Elapsed Time: 0.08890819549560547
torch.Size([1, 1000])
tensor([[ 4.8389e+00,  2.0648e+01, -7.9396e-01,  1.6835e+00,  3.2860e+00,
          3.8289e+00,  3.3749e+00,  4.6865e+00,  3.4357e+00, -3.1508e+00,
          2.7219e+00,  2.0141e+00,  4.9674e+00, -1.9659e-01,  2.5462e+00,
          3.9514e+00,  1.0493e+00,  1.3478e+00, -1.9551e+00,  1.4296e-01,
         -6.3565e-02,  1.5156e+00, -1.8726e+00, -1.7927e-01, -1.8498e+00,
          7.3188e+00,  1.4851e+01,  1.7775e+01,  6.8965e+00,  1.6232e+01,
          4.2191e+00,  8.4351e+00,  9.0231e+00,  4.7880e+00,  3.6737e+00,
          5.1464e+00,  5.3828e+00,  3.0542e+00,  9.8607e+00,  3.7158e+00,
          1.0031e+01,  6.6635e+00,  1.2615e+01,  6.7393e+00,  7.8229e+00,
          3.1481e+00,  8.2788e+00,  5.9477e+00,  3.5756e+00,  4.9702e+00,
          3.5887e+00,  1.7237e+00,  7.7454e+00,  5.6308e+00,  5.2948e+00,
          3.9757e+00,  4.9697e+00,  3.3797e+00,  4.9800e+00,  5.3257e+00,
          5.9610e+00,  5.5498e+00,  4.3237e+00,  4.6241e

       grad_fn=<AddmmBackward>)
tensor([ 1, 27, 29, 26, 42])
